In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
import seaborn as sns
import pandas as pd
import re
import json
import sqlite3
sns.set_style('darkgrid')

In [ ]:
import sys
sys.path.append('../../pytpc')
import pytpc

# Output database connection

In [ ]:
gasdb = sqlite3.connect('../data/gases/gasdata.db')
gdb_curs = gasdb.cursor()

# Read data from ASTAR data files

In [ ]:
def read_astar(fp):
    return pd.read_table(fp, delim_whitespace=True, skiprows=5, names=('energy', 'dedx', 'range'), index_col=0)

## Helium data

This is for pure helium gas

In [ ]:
he_data = read_astar('../data/raw/helium_astar.txt')

In [ ]:
he_data.to_sql('helium', gasdb, if_exists='replace')

In [ ]:
gdb_curs.execute('INSERT INTO masses VALUES ("helium", 4.002)')
gasdb.commit()

In [ ]:
hegas = pytpc.gases.InterpolatedGas('helium', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
hegas = pytpc.gases.InterpolatedGas('helium', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.range(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')

## CO2 data

This is for pure CO2 gas

In [ ]:
co2_data = read_astar('../data/raw/co2_astar.txt')

In [ ]:
co2_data.to_sql('carbon_dioxide', gasdb, if_exists='replace')

In [ ]:
gdb_curs.execute('INSERT INTO Masses VALUES ("carbon_dioxide", 44.01)')
gasdb.commit()

In [ ]:
co2gas = pytpc.gases.InterpolatedGas('carbon_dioxide', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, co2gas.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
co2gas = pytpc.gases.InterpolatedGas('carbon_dioxide', 760)
ens = np.logspace(-3, 3, 100)
plt.plot(ens, co2gas.range(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')

# Gas Mixtures

## Helium-CO2 90/10

In [ ]:
heco2 = pytpc.gases.InterpolatedGasMixture(760, ('helium', 0.9), ('carbon_dioxide', 0.1))
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.energy_loss(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.range(ens, 4, 2))
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2), label='Pure He')
plt.plot(ens, co2gas.energy_loss(ens, 4, 2), label='Pure CO2')
plt.plot(ens, heco2.energy_loss(ens, 4, 2), label='He-CO2 90/10')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('dE/dx at 760 torr [MeV/m]')
plt.legend()

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.energy_loss(ens, 4, 2)/co2gas.energy_loss(ens, 4, 2))
plt.loglog()

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, hegas.range(ens, 4, 2), label='Pure He')
plt.plot(ens, co2gas.range(ens, 4, 2), label='Pure CO2')
plt.plot(ens, heco2.range(ens, 4, 2), label='He-CO2 90/10')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range at 760 torr [m]')
plt.legend(loc=2)
# plt.savefig('/Users/josh/Desktop/range_fullpress.pdf', transparent=True)

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.components[0][0].energy_loss(ens, 4, 2), 
         label='Pure Helium at {} torr'.format(heco2.components[0][1] * 760))
plt.plot(ens, heco2.components[1][0].energy_loss(ens, 4, 2), 
         label='Pure CO2 at {} torr'.format(heco2.components[1][1] * 760))
plt.plot(ens, heco2.energy_loss(ens, 4, 2), label='He-CO2 90/10 at 760 torr')
plt.loglog()
plt.xlabel('de/dx [MeV/m]')
plt.ylabel('Range [m]')
plt.legend(loc=1)

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, heco2.components[0][0].range(ens, 4, 2), 
         label='Pure Helium at {} torr'.format(heco2.components[0][1] * 760))
plt.plot(ens, heco2.components[1][0].range(ens, 4, 2), 
         label='Pure CO2 at {} torr'.format(heco2.components[1][1] * 760))
plt.plot(ens, heco2.range(ens, 4, 2), label='He-CO2 90/10 at 760 torr')
plt.loglog()
plt.xlabel('Energy [MeV]')
plt.ylabel('Range [m]')
plt.legend(loc=2)

In [ ]:
plt.plot(ens, heco2.components[0][0].energy_loss(ens, 4, 2) / heco2.components[1][0].energy_loss(ens, 4, 2))
plt.semilogx()

In [ ]:
ens = np.logspace(-3, 3, 100)
ranges = heco2.range(ens, 4, 2)
heco2_table = pd.DataFrame(np.vstack((ens, ranges)).T, columns=('energy', 'range'))

In [ ]:
heco2_table.to_csv('/Users/josh/Desktop/heco2_range_table.csv', index=False)